In [ ]:
import pandas as pd
import pymongo
import numpy as np 

DB_CONN_STRING = "mongodb://localhost:27017/"
DB_NAME = "recent-poland-covid-vaccine-tweets"
COLLECTION_NAME = "tweets"

mongo_client = pymongo.MongoClient(DB_CONN_STRING)
database_name = mongo_client[DB_NAME]
data_collection = database_name[COLLECTION_NAME]

projection = {"_id": 1, "quotedTweet":1, "sentiment": 1}
tweets = [tweet for tweet in data_collection.find({'quotedTweet' : {'$gte': 1}}, projection)]

df = pd.DataFrame(tweets)
# df

In [ ]:
df

In [ ]:
def get_sentiment_by_id(_id):
    projection = {"_id": 1, "sentiment": 1}
    return [tweet['sentiment'] for tweet in data_collection.find({'_id' : _id}, projection)][0]

df['quotedTweetSentiment'] = df.apply(lambda row: get_sentiment_by_id(row['quotedTweet']), axis=1)
df

In [ ]:
df2 = df[["sentiment", "quotedTweetSentiment"]]
df2["count"] = np.ones(len(df2['sentiment']))
df2 = df2.groupby(by=['sentiment', 'quotedTweetSentiment']).sum()
df2["sentiment"] = df2.index.to_frame()["sentiment"]
df2["quotedTweetSentiment"] = df2.index.to_frame()["quotedTweetSentiment"]

df2["sentiment"] = df2.apply(lambda row: float(row["sentiment"]), axis=1)
df2["quotedTweetSentiment"] = df2.apply(lambda row: float(row["quotedTweetSentiment"]), axis=1)

df2

In [ ]:
df3 = df2.pivot(index='sentiment',columns='quotedTweetSentiment').fillna(0)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(15,10))
ax = sns.heatmap(df3)

In [ ]:
df3.loc[[0.,],[('count',  0.0)]] = 0

In [ ]:
f, ax = plt.subplots(figsize=(15,10))
ax = sns.heatmap(df3)